In [8]:
import pandas as pd

In [9]:
def read_data(file):
    df = pd.read_excel(f"../data/death_rate/{file}")
    # df = df.drop(labels=["Unnamed: 1", "Unnamed: 2"], axis=1)
    df = df.rename(columns={"Unnamed: 0": "region"})
    df_long = df.melt(
        id_vars='region',
        var_name='date',
        value_name=f'{file[:-4]}'
    )
    df_long['date'] = df_long['date'].astype(int)

    df_long = df_long.sort_values(['region', 'date'])
    df_long = df_long.dropna(subset=f'{file[:-4]}')
    df_long.region = df_long.region.str.strip()
    return df_long

In [10]:
import os

df_final = pd.DataFrame()
for filename in [file for file in os.listdir("../data/death_rate/") if "csv" not in file]:
    print(filename)
    dff = read_data(filename)
    if df_final.empty:
        df_final = dff
    else:
        df_final = pd.merge(df_final, dff, on=["date", "region"], how="inner")

death_rate_village.xls
death_rate_city.xls
death_rate_all.xls


In [11]:
lag_steps = [15, 10, 5, 3]
factors = df_final.drop(labels=["region", "date"], axis=1).columns.tolist()

for factor in factors:
    for lag in lag_steps:
        df_final[f'{factor}_lag_{lag}'] = df_final.groupby('region')[factor].shift(lag)

In [12]:
df_final

,region,date,death_rate_village,death_rate_city,death_rate_all,death_rate_village_lag_15,death_rate_village_lag_10,death_rate_village_lag_5,death_rate_village_lag_3,death_rate_city_lag_15,death_rate_city_lag_10,death_rate_city_lag_5,death_rate_city_lag_3,death_rate_all_lag_15,death_rate_all_lag_10,death_rate_all_lag_5,death_rate_all_lag_3
0,Архангельская область (без АО),2012,18.0,12.5,13.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Архангельская область (без АО),2015,18.2,12.3,13.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Архангельская область (без АО),2016,18.5,12.3,13.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Архангельская область (кроме Ненецкого автоном...,2017,18.1,11.8,13.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Архангельская область (кроме Ненецкого автоном...,2018,18.3,11.9,13.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,Российская Федерация,2018,13.6,12.0,12.4,18.4,16.7,14.5,14.4,15.6,13.8,12.5,12.6,16.4,14.5,13.0,13.0
2141,Российская Федерация,2019,13.3,11.9,12.3,18.1,16.3,14.5,14.2,15.2,13.3,12.6,12.4,15.9,14.1,13.1,12.9
2142,Российская Федерация,2020,15.4,14.3,14.6,18.6,16.1,14.4,13.7,15.1,13.5,12.6,12.0,16.1,14.2,13.0,12.4
2143,Российская Федерация,2021,17.5,16.5,16.7,17.4,15.2,14.2,13.6,14.3,12.8,12.4,12.0,15.1,13.5,12.9,12.4


In [13]:
df_final.to_csv("../data/death_rate/death_rate_features.csv", index=False)